---
draft: true 
date: 2025-01-08
description: "Calculating adjusted xG coefficients"
slug: adjusted-xg
---

# **Calculating score- and venue-adjusted xG coefficients**

---

## **Intro**

Intro language here

---

## **Housekeeping**

### Import dependencies

Import the dependencies we'll need to simulate today's games

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

import chickenstats

from chickenstats.api import ChickenStats
from chickenstats.utilities import ChickenProgress
import chickenstats.utilities
from chickenstats.chicken_nhl import Season, Scraper
from chickenstats.chicken_nhl.scrape import Game

from rich.progress import track

from hockey_rink import NHLRink
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import matplotlib.patheffects as mpe
import matplotlib.ticker as ticker
import seaborn as sns

from dotenv import load_dotenv
from pathlib import Path

import datetime as dt

import math

from typing import Optional

### Pandas options

Set different pandas options

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Load environment variables

Load environment variables to access the chickenstats API

In [ ]:
env_file = Path("../../.env")
load_dotenv(env_file)

### Chickenstats matplotlib style

chickenstats.utilities includes a custom style package - this activates it

In [ ]:
plt.style.use("chickenstats")

---

## **Load data**

In [ ]:
concat_list = []

years = list(range(2023, 2009, -1))

for year in track(years):
    file_path = Path(
        f"../../../../chickenstats-api/data/chickenstats/raw/pbp/pbp_{year}.csv"
    )
    raw_df = pd.read_csv(file_path, low_memory=False)

    events = ["GOAL", "SHOT", "MISS", "BLOCK"]
    raw_df = raw_df.loc[raw_df.event.isin(events)]

    concat_list.append(raw_df)

In [ ]:
shot_events_df = pd.concat(concat_list, ignore_index=True)

In [ ]:
def prepare_shots_data(play_by_play: pd.DataFrame, strength_state: str) -> pd.DataFrame:
    """Docstring."""
    condition = play_by_play.strength_state == strength_state
    shots_data = play_by_play.loc[condition].reset_index(drop=True)

    shots_data["strength_state2"] = strength_state

    condition = shots_data.block == 1
    shots_data.event_team = np.where(
        condition, shots_data.opp_team, shots_data.event_team
    )

    shots_data.block = shots_data.block + shots_data.teammate_block

    conditions = [shots_data.home_score_diff < -3, shots_data.home_score_diff > 3]
    values = [-3, 3]
    shots_data.home_score_diff = np.select(
        conditions, values, shots_data.home_score_diff
    )

    shots_data.is_home = np.where(shots_data.event_team == shots_data.home_team, 1, 0)

    group_list = ["strength_state2", "home_score_diff", "is_home"]
    agg_stats = {x: "sum" for x in ["goal", "pred_goal", "shot", "miss", "block"]}
    shots_data = shots_data.groupby(group_list, as_index=False).agg(agg_stats)

    shots_data["fenwick"] = shots_data[["goal", "shot", "miss"]].sum(axis=1)
    shots_data["corsi"] = shots_data[["goal", "shot", "miss", "block"]].sum(axis=1)

    agg_stats = {}

    for stat in ["goal", "pred_goal", "shot", "miss", "block", "fenwick", "corsi"]:
        shots_data[f"home_{stat}"] = np.where(
            shots_data.is_home == 1, shots_data[stat], 0
        )
        shots_data[f"away_{stat}"] = np.where(
            shots_data.is_home == 0, shots_data[stat], 0
        )

        agg_stats.update({f"home_{stat}": "sum", f"away_{stat}": "sum"})

    shots_data = (
        shots_data.groupby(["strength_state2", "home_score_diff"], as_index=False)
        .agg(agg_stats)
        .rename(columns={"strength_state2": "strength_state"})
    )

    return shots_data

In [ ]:
def calculate_weights(shots_data: pd.DataFrame) -> pd.DataFrame:
    """Docstring."""
    shots_data = shots_data.copy()

    strength_state = shots_data.strength_state.unique()[0]

    if strength_state in ["5v5", "4v4"]:
        weight_columns = ["fenwick", "corsi"]

        if strength_state == "5v5":
            weight_columns.insert(0, "shot")

        for weight_column in weight_columns:
            mean_values = shots_data[
                [f"home_{weight_column}", f"away_{weight_column}"]
            ].mean(axis=1)

            shots_data[f"home_{weight_column}_weight"] = (
                mean_values / shots_data[f"home_{weight_column}"]
            )
            shots_data[f"away_{weight_column}_weight"] = (
                mean_values / shots_data[f"away_{weight_column}"]
            )

    if strength_state in ["5v5", "4v4", "5v4", "5v3", "4v3", "1v0"]:
        if strength_state == "5v5":
            weight_columns = ["goal", "pred_goal"]

        if strength_state == "4v4":
            weight_columns = ["shot"]

        if strength_state in ["5v4", "5v3", "4v3"]:
            weight_columns = ["goal", "pred_goal", "shot", "fenwick", "corsi"]

        if strength_state == "1v0":
            weight_columns = ["goal"]

        weight_groups = ["trailing", "tied", "leading"]
        weight_group_conditions = [
            shots_data.home_score_diff < 0,
            shots_data.home_score_diff == 0,
            shots_data.home_score_diff > 0,
        ]

        shots_data["score_diff_group"] = np.select(
            weight_group_conditions, weight_groups, default=""
        )

        for weight_column in weight_columns:
            group_mean = (
                shots_data.groupby("score_diff_group")[
                    [f"home_{weight_column}", f"away_{weight_column}"]
                ]
                .transform("sum")[[f"home_{weight_column}", f"away_{weight_column}"]]
                .sum(axis=1)
            ) / 2

            group_home = shots_data.groupby("score_diff_group")[
                f"home_{weight_column}"
            ].transform("sum")

            group_away = shots_data.groupby("score_diff_group")[
                f"away_{weight_column}"
            ].transform("sum")

            shots_data[f"home_{weight_column}_weight"] = group_mean / group_home
            shots_data[f"away_{weight_column}_weight"] = group_mean / group_away

    if strength_state in ["4v4", "3v3"]:
        if strength_state == "4v4":
            weight_columns = ["goal", "pred_goal"]

        if strength_state == "3v3":
            weight_columns = ["goal", "pred_goal", "shot", "fenwick", "corsi"]

        for weight_column in weight_columns:
            group_mean = (
                shots_data.groupby("strength_state")[
                    [f"home_{weight_column}", f"away_{weight_column}"]
                ]
                .transform("sum")[[f"home_{weight_column}", f"away_{weight_column}"]]
                .sum(axis=1)
            ) / 2

            group_home = shots_data.groupby("strength_state")[
                f"home_{weight_column}"
            ].transform("sum")

            group_away = shots_data.groupby("strength_state")[
                f"away_{weight_column}"
            ].transform("sum")

            shots_data[f"home_{weight_column}_weight"] = group_mean / group_home
            shots_data[f"away_{weight_column}_weight"] = group_mean / group_away

    return shots_data

In [ ]:
concat_list = []

strength_states = ["5v5", "4v4", "3v3", "5v4", "5v3", "4v3", "1v0"]

for strength_state in strength_states:
    shots_data = prepare_shots_data(
        play_by_play=shot_events_df, strength_state=strength_state
    )

    weights = calculate_weights(shots_data=shots_data)

    concat_list.append(weights)

In [ ]:
test = pd.concat(concat_list, ignore_index=True)

In [ ]:
columns = [
    "strength_state",
    "home_score_diff",
    "score_diff_group",
    "home_goal_weight",
    "away_goal_weight",
    "home_pred_goal_weight",
    "away_pred_goal_weight",
    "home_shot_weight",
    "away_shot_weight",
    "home_fenwick_weight",
    "away_fenwick_weight",
    "home_corsi_weight",
    "away_corsi_weight",
    "home_goal",
    "away_goal",
    "home_pred_goal",
    "away_pred_goal",
    "home_shot",
    "away_shot",
    "home_miss",
    "away_miss",
    "home_block",
    "away_block",
    "home_fenwick",
    "away_fenwick",
    "home_corsi",
    "away_corsi",
]

weights_dict = test[columns].to_dict(orient="records")

In [ ]:
weights_dict = {}

strength_states = test.strength_state.unique().tolist()

for strength_state in strength_states:
    score_states = (
        test.loc[test.strength_state == strength_state]
        .home_score_diff.unique()
        .tolist()
    )

    score_state_dict = {}

    for score_state in score_states:
        conditions = np.logical_and(
            test.strength_state == strength_state, test.home_score_diff == score_state
        )
        data = test.loc[conditions].iloc[0]

        columns = [
            "score_diff_group",
            "home_goal_weight",
            "away_goal_weight",
            "home_pred_goal_weight",
            "away_pred_goal_weight",
            "home_shot_weight",
            "away_shot_weight",
            "home_fenwick_weight",
            "away_fenwick_weight",
            "home_corsi_weight",
            "away_corsi_weight",
            "home_goal",
            "away_goal",
            "home_pred_goal",
            "away_pred_goal",
            "home_shot",
            "away_shot",
            "home_miss",
            "away_miss",
            "home_block",
            "away_block",
            "home_fenwick",
            "away_fenwick",
            "home_corsi",
            "away_corsi",
        ]

        score_state_dict.update({score_state: {x: data[x] for x in columns}})

    strength_state_dict = {strength_state: score_state_dict}

    weights_dict.update(strength_state_dict)

In [ ]:
import pickle

In [ ]:
with open("score_adjustments.pkl", "wb") as f:
    pickle.dump(weights_dict, f)

In [ ]:
with open("score_adjustments.pkl", "rb") as f:
    test_pickle = pickle.load(f)

In [ ]:
season = Season(2024)
schedule = season.schedule("NSH")

game_ids = schedule.loc[schedule.game_state == "OFF"].game_id.tolist()
game_id = game_ids[0]

In [ ]:
scraper = Scraper(game_ids)

In [ ]:
pbp = scraper.play_by_play

In [ ]:
scraper.prep_stats(level="season")
stats = scraper.stats.copy()

In [ ]:
pbp.pred_goal_adj.max()

In [ ]:
stats.columns.tolist()